# Введение в MapReduce модель на Python


In [140]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [141]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [142]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [143]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [144]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [145]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [146]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [147]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [148]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [149]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [150]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [151]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [152]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [153]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [154]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.0561552965469185)),
 (1, np.float64(2.0561552965469185)),
 (2, np.float64(2.0561552965469185)),
 (3, np.float64(2.0561552965469185)),
 (4, np.float64(2.0561552965469185))]

## Inverted index 

In [155]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [156]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [157]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [158]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18)]),
 (1, [('banana', 2), ('what', 10)])]

## TeraSort

In [159]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.031935634240972566)),
   (None, np.float64(0.11691246582267845)),
   (None, np.float64(0.11945652156292264)),
   (None, np.float64(0.22033490461423222)),
   (None, np.float64(0.2410633880277554)),
   (None, np.float64(0.2841802326661095)),
   (None, np.float64(0.29174048456693147)),
   (None, np.float64(0.398736028816858)),
   (None, np.float64(0.42786023620542857)),
   (None, np.float64(0.43789146608740215)),
   (None, np.float64(0.46212232533625675)),
   (None, np.float64(0.4644151669946819)),
   (None, np.float64(0.4833865981740264)),
   (None, np.float64(0.4929610017757182))]),
 (1,
  [(None, np.float64(0.5079252789090506)),
   (None, np.float64(0.5209030504089659)),
   (None, np.float64(0.5248042105965424)),
   (None, np.float64(0.5628209825514109)),
   (None, np.float64(0.6020911211182765)),
   (None, np.float64(0.6458411166548984)),
   (None, np.float64(0.679454581831782)),
   (None, np.float64(0.6870939503947159)),
   (None, np.float64(0.729726299750

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [160]:
import random
from itertools import groupby

In [161]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER)))))

In [162]:
def MAP(key, value):
    yield key, value

def REDUCE(key, values):
    yield max(values)

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

array = list(RECORDREADER(20))
print("Input Data:", [v for _, v in array])

result = list(MapReduce(array, MAP, REDUCE))
print("Max Value:", result[0])

Input Data: [54, 86, 81, 51, 13, 87, 78, 48, 86, 93, 76, 48, 32, 37, 25, 37, 68, 83, 47, 8]
Max Value: 93


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [163]:
def MAP(key, value):
  yield None, value

def REDUCE(key, values):
  total = sum(values)
  count = len(values)
  yield total / count

def RECORDREADER(count):
    return ((None, random.randint(0, 100)) for _ in range(count))

array = list(RECORDREADER(100))
print("Input Data:", [v for _, v in array])

result = list(MapReduce(array, MAP, REDUCE))
print("Average value:", result[0])

Input Data: [95, 66, 6, 42, 40, 23, 59, 67, 56, 60, 11, 97, 21, 69, 73, 69, 25, 98, 34, 24, 97, 90, 77, 13, 29, 44, 6, 91, 91, 83, 13, 23, 42, 70, 58, 20, 40, 49, 14, 50, 76, 0, 68, 62, 68, 0, 0, 50, 64, 15, 57, 25, 48, 66, 48, 81, 64, 3, 3, 17, 93, 91, 69, 48, 17, 2, 50, 94, 85, 58, 72, 62, 49, 36, 74, 59, 61, 52, 22, 75, 32, 25, 92, 37, 15, 15, 10, 68, 54, 8, 37, 57, 83, 31, 61, 31, 53, 87, 7, 83]
Average value: 49.05


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [164]:
import random
from itertools import groupby

def groupByKey_sorted(iterable):
    sorted_iterable = sorted(iterable, key=lambda x: x[0])
    return ((key, [v for _, v in group]) for key, group in groupby(sorted_iterable, key=lambda x: x[0]))

def MAP(key, value):
    yield 1, value

def REDUCE(key, values):
    total = sum(values)
    count = len(values)
    yield total / count

def RECORDREADER(count):
    return ((1, random.randint(0, 100)) for _ in range(count))

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupByKey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER)))))

array = list(RECORDREADER(100))
print("Input Data:", array)

result = list(MapReduce(array, MAP, REDUCE))
print("Average value:", result[0])

Input Data: [(1, 32), (1, 1), (1, 13), (1, 11), (1, 33), (1, 72), (1, 45), (1, 55), (1, 68), (1, 90), (1, 98), (1, 8), (1, 13), (1, 17), (1, 36), (1, 59), (1, 90), (1, 80), (1, 89), (1, 48), (1, 5), (1, 94), (1, 88), (1, 28), (1, 18), (1, 0), (1, 6), (1, 13), (1, 2), (1, 9), (1, 91), (1, 89), (1, 22), (1, 77), (1, 93), (1, 8), (1, 9), (1, 89), (1, 77), (1, 40), (1, 66), (1, 98), (1, 36), (1, 26), (1, 57), (1, 17), (1, 10), (1, 10), (1, 94), (1, 60), (1, 68), (1, 68), (1, 18), (1, 9), (1, 38), (1, 89), (1, 88), (1, 24), (1, 51), (1, 24), (1, 33), (1, 23), (1, 79), (1, 73), (1, 40), (1, 63), (1, 36), (1, 63), (1, 72), (1, 100), (1, 20), (1, 54), (1, 64), (1, 2), (1, 10), (1, 61), (1, 50), (1, 4), (1, 21), (1, 51), (1, 26), (1, 6), (1, 96), (1, 69), (1, 45), (1, 76), (1, 1), (1, 90), (1, 24), (1, 75), (1, 72), (1, 69), (1, 99), (1, 82), (1, 38), (1, 19), (1, 47), (1, 98), (1, 93), (1, 70)]
Average value: 48.81


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [165]:
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a dog"""

documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                    reduce_partition[1]))), reduce_partitions)
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])
    print(f"{total_values_sent} key-value pairs were sent over a network.")
    return reduce_outputs

def INPUTFORMAT():
    global maps
    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)
    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, word)

def REDUCE(key: str, value):
    yield key

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'a', 'is', 'it']), (1, ['dog', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [166]:
def MAP(el_list):
    return [(t, [t]) for t in el_list if C(t)]

def REDUCE(mapped_items):
    return [value for sublist in mapped_items for value in sublist]

def C(t):
    return t[0] % 2 == 0

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

record = RECORDREADER(5)  # Генерация данных
print("Input data:", record)

part_size = 5
record_partitioned = [record[i:i + part_size] for i in range(0, len(record), part_size)]
print("Partitioned data:", record_partitioned)

mapped_results = list(map(MAP, record_partitioned))
reduced_result = REDUCE(mapped_results)

print("Filtered result:", reduced_result)


Input data: [(99, 80, 73), (64, 31, 74), (80, 20, 40), (65, 50, 86), (42, 54, 26)]
Partitioned data: [[(99, 80, 73), (64, 31, 74), (80, 20, 40), (65, 50, 86), (42, 54, 26)]]
Filtered result: [((64, 31, 74), [(64, 31, 74)]), ((80, 20, 40), [(80, 20, 40)]), ((42, 54, 26), [(42, 54, 26)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [167]:
from typing import Iterator, NamedTuple

S = {25, 90, 69}

def MAP(t):
    filtered = tuple(el for el in t if el in S)
    return (filtered, filtered)

def REDUCE(key, values: Iterator[NamedTuple]):
    return key, key

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(5)
print("Input records:", record)

map_output = list(map(MAP, RECORDREADER(100)))
print("MAP output:", map_output)

shuffle_output = group_by_key(map(MAP, RECORDREADER(100)))

reduce_output = list(map(lambda x: REDUCE(*x)[0], list(shuffle_output)))
print("Reduce output:", reduce_output)

Input records: [(93, 87, 91), (97, 30, 78), (56, 95, 39), (9, 92, 95), (88, 89, 55)]
MAP output: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((),

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [168]:
import random
from collections import defaultdict

def MAP(t):
    return t, t

def REDUCE(key, values):
    return key, key

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

record = RECORDREADER(100)
print("Input records:", record)

map_output = map(MAP, record)
print("MAP output:", list(map_output))

shuffle_output = group_by_key(map(MAP, record))
print("Shuffle output:", list(shuffle_output))

reduce_output = [REDUCE(k, v) for k, v in shuffle_output]
print("Reduce output:", reduce_output)

Input records: [(12, 7, 92), (72, 31, 8), (7, 74, 19), (7, 43, 100), (66, 64, 37), (5, 88, 64), (80, 31, 61), (27, 37, 55), (99, 87, 67), (97, 46, 90), (20, 51, 30), (51, 0, 94), (49, 9, 11), (54, 29, 6), (74, 6, 65), (72, 60, 34), (99, 44, 34), (97, 7, 75), (5, 78, 57), (44, 100, 65), (79, 55, 44), (99, 88, 77), (64, 13, 39), (25, 48, 8), (26, 89, 25), (51, 71, 62), (9, 100, 55), (23, 64, 66), (32, 55, 12), (67, 58, 78), (100, 16, 67), (53, 78, 32), (86, 65, 44), (57, 20, 100), (4, 83, 58), (7, 35, 23), (27, 15, 7), (78, 45, 22), (45, 50, 21), (44, 99, 8), (13, 53, 21), (95, 1, 95), (80, 1, 98), (79, 97, 35), (2, 38, 19), (9, 51, 70), (15, 32, 17), (4, 49, 50), (28, 47, 79), (64, 47, 23), (26, 12, 38), (58, 11, 52), (91, 84, 66), (81, 76, 42), (77, 12, 9), (20, 27, 55), (47, 36, 62), (87, 32, 99), (6, 10, 82), (86, 80, 76), (22, 52, 87), (4, 18, 72), (71, 29, 74), (67, 97, 16), (38, 69, 95), (75, 42, 22), (64, 62, 13), (68, 32, 85), (93, 52, 16), (27, 49, 70), (37, 14, 6), (75, 32, 35

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [169]:
def MAP(t):
    return t, t

def REDUCE(key, values):
    if len(values) == 2:
        return key, key
    return None

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)
print("Input records:", record)

map_output = list(map(MAP, record))
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = list(filter(None, map(lambda x: REDUCE(*x), shuffle_output)))
print("Reduce output:", reduce_output)

Input records: [(2, 3), (3, 2), (0, 0), (3, 0), (2, 1), (0, 1), (3, 1), (0, 3), (0, 1), (1, 0), (0, 3), (2, 1), (1, 3), (2, 1), (1, 1), (2, 3), (2, 3), (0, 0), (1, 2), (3, 0), (0, 2), (3, 0), (3, 1), (1, 3), (3, 1), (1, 2), (1, 0), (1, 0), (3, 0), (2, 3), (0, 0), (0, 1), (0, 2), (0, 1), (0, 1), (0, 3), (2, 1), (1, 3), (3, 1), (0, 0), (2, 3), (0, 2), (0, 2), (1, 1), (1, 3), (0, 2), (1, 1), (1, 0), (0, 0), (3, 2), (3, 2), (1, 2), (0, 1), (1, 2), (2, 0), (1, 3), (2, 2), (0, 3), (2, 3), (2, 1), (1, 0), (1, 2), (0, 3), (0, 2), (3, 2), (2, 1), (0, 2), (1, 2), (3, 0), (2, 1), (0, 3), (0, 1), (3, 3), (3, 2), (1, 2), (2, 3), (3, 2), (2, 3), (1, 3), (3, 2), (2, 3), (3, 1), (1, 1), (2, 3), (1, 0), (2, 2), (3, 0), (3, 2), (2, 1), (0, 0), (1, 2), (1, 2), (0, 2), (1, 0), (2, 0), (1, 0), (3, 3), (0, 1), (3, 3), (2, 0)]
MAP output: [((2, 3), (2, 3)), ((3, 2), (3, 2)), ((0, 0), (0, 0)), ((3, 0), (3, 0)), ((2, 1), (2, 1)), ((0, 1), (0, 1)), ((3, 1), (3, 1)), ((0, 3), (0, 3)), ((0, 1), (0, 1)), ((1, 0), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [170]:
import random

rels = [1, 2]

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(count))
    rel_id = random.choice(rels)
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple(3) for _ in range(count)]

def MAP(t: Tuple):
    return t.data, t.rel_id

def REDUCE(key, values):
    if rels[0] in values:
        return key, key
    return None

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)

map_output = list(map(MAP, record))
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = list(filter(None, map(lambda x: REDUCE(*x), shuffle_output)))
print("Reduce output:", reduce_output)

MAP output: [(((3, 3), (1, 0), (1, 2)), 2), (((1, 2), (2, 2), (1, 3)), 1), (((1, 1), (1, 0), (0, 1)), 2), (((1, 0), (0, 3), (2, 2)), 2), (((0, 1), (2, 2), (1, 2)), 1), (((3, 3), (0, 2), (0, 0)), 2), (((2, 1), (1, 2), (1, 1)), 1), (((3, 2), (2, 3), (1, 3)), 2), (((1, 0), (1, 1), (1, 0)), 2), (((3, 1), (0, 2), (2, 2)), 1), (((0, 3), (3, 3), (2, 0)), 2), (((0, 3), (3, 2), (1, 0)), 2), (((3, 0), (1, 3), (0, 2)), 1), (((1, 1), (0, 3), (3, 3)), 2), (((1, 0), (3, 0), (0, 0)), 2), (((2, 0), (3, 0), (2, 3)), 1), (((3, 2), (1, 0), (1, 0)), 1), (((0, 1), (0, 3), (3, 1)), 1), (((2, 3), (1, 0), (2, 1)), 1), (((1, 1), (1, 0), (3, 1)), 1), (((2, 1), (1, 2), (1, 0)), 2), (((3, 0), (2, 1), (2, 1)), 2), (((1, 1), (0, 1), (1, 0)), 1), (((2, 0), (0, 2), (1, 2)), 2), (((3, 1), (1, 0), (0, 2)), 1), (((2, 2), (0, 0), (0, 1)), 2), (((3, 0), (1, 2), (0, 2)), 2), (((2, 3), (0, 3), (1, 3)), 1), (((3, 2), (3, 1), (0, 2)), 1), (((0, 0), (3, 3), (0, 2)), 1), (((2, 3), (2, 3), (1, 1)), 2), (((3, 1), (1, 0), (0, 2)),

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [171]:
import random
from collections import defaultdict

rels = [1, 2]  # Возможные идентификаторы связей

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple():
    return Tuple((random.randint(0, 3), random.randint(0, 3)), random.choice(rels))

def RECORDREADER(count):
    return [get_random_tuple() for _ in range(count)]

def MAP(t: Tuple):
    return (t.data[1], (t.rel_id, t.data[0])) if t.rel_id == rels[0] else (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values):
    return [(v[0], key, v[1]) for v in values]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)

map_output = list(map(MAP, record))
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [item for sublist in map(lambda x: REDUCE(*x), shuffle_output) for item in sublist]
print("Reduce output:", reduce_output)


MAP output: [(3, (2, 2)), (3, (2, 3)), (2, (2, 3)), (2, (2, 3)), (0, (2, 3)), (0, (1, 1)), (1, (1, 0)), (2, (1, 0)), (1, (2, 2)), (1, (1, 2)), (0, (2, 1)), (3, (1, 0)), (3, (2, 1)), (1, (1, 3)), (1, (2, 3)), (1, (1, 1)), (3, (2, 1)), (3, (1, 2)), (2, (1, 0)), (3, (1, 2)), (2, (1, 2)), (3, (1, 1)), (0, (2, 3)), (0, (2, 3)), (0, (2, 0)), (1, (2, 3)), (0, (1, 1)), (2, (1, 3)), (2, (2, 0)), (1, (1, 2)), (2, (2, 0)), (2, (2, 0)), (0, (1, 1)), (1, (2, 0)), (0, (1, 3)), (0, (1, 3)), (0, (1, 3)), (3, (2, 0)), (2, (2, 1)), (1, (1, 3)), (0, (1, 2)), (0, (2, 0)), (3, (1, 2)), (1, (1, 1)), (3, (2, 0)), (1, (1, 0)), (1, (2, 3)), (3, (1, 2)), (2, (1, 2)), (1, (2, 3)), (1, (1, 1)), (1, (1, 0)), (1, (2, 0)), (1, (2, 1)), (1, (1, 2)), (0, (2, 3)), (1, (2, 3)), (1, (2, 0)), (1, (2, 3)), (1, (2, 3)), (1, (2, 2)), (3, (1, 0)), (2, (1, 0)), (3, (1, 1)), (3, (1, 1)), (2, (1, 2)), (1, (1, 0)), (3, (2, 3)), (2, (2, 3)), (2, (2, 2)), (0, (2, 2)), (3, (1, 2)), (0, (1, 0)), (0, (2, 3)), (3, (2, 3)), (2, (1, 3)),

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [172]:
import random

def get_random_tuple():
    return tuple(random.choices(range(4), k=3))

def RECORDREADER(count):
    return [get_random_tuple() for _ in range(count)]

def MAP(t):
    return t[:2]

def REDUCE(key, values):
    return key, sum(values)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)

map_output = [MAP(t) for t in record]
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [REDUCE(k, v) for k, v in shuffle_output]
print("Reduce output:", reduce_output)


MAP output: [(2, 0), (1, 2), (0, 2), (3, 0), (3, 3), (0, 3), (3, 2), (1, 3), (2, 3), (2, 1), (0, 2), (0, 3), (3, 3), (3, 1), (0, 3), (3, 2), (2, 1), (3, 1), (3, 2), (3, 0), (2, 3), (0, 3), (1, 3), (2, 1), (0, 1), (3, 0), (0, 1), (3, 0), (3, 1), (0, 2), (1, 2), (0, 3), (0, 3), (2, 3), (1, 0), (3, 0), (3, 3), (3, 0), (2, 0), (1, 1), (0, 0), (1, 1), (0, 2), (1, 2), (1, 0), (0, 0), (2, 3), (0, 2), (2, 1), (0, 2), (0, 2), (0, 1), (3, 1), (0, 0), (3, 2), (2, 2), (3, 3), (0, 1), (1, 2), (3, 1), (1, 0), (3, 2), (3, 0), (1, 0), (1, 2), (0, 3), (1, 3), (3, 1), (2, 2), (3, 1), (0, 0), (1, 3), (2, 0), (3, 1), (3, 2), (3, 3), (2, 0), (2, 1), (1, 2), (0, 1), (0, 2), (2, 3), (2, 3), (3, 2), (1, 3), (3, 2), (1, 0), (0, 1), (2, 0), (3, 2), (0, 1), (3, 3), (3, 3), (1, 2), (1, 0), (3, 2), (1, 3), (0, 0), (0, 1), (1, 2)]
Shuffle output: [(2, [0, 3, 1, 1, 3, 1, 3, 0, 3, 1, 2, 2, 0, 0, 1, 3, 3, 0]), (1, [2, 3, 3, 2, 0, 1, 1, 2, 0, 2, 0, 0, 2, 3, 3, 2, 3, 0, 2, 0, 3, 2]), (0, [2, 3, 2, 3, 3, 3, 1, 1, 2, 3, 3

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [173]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [174]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield (j, k), big_mat[j,k]
      
def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(small_mat.shape[0]):
        yield (i, k), w * small_mat[i][j]

def REDUCE(key, values):
   (i, k) = key
   el_value = sum(values)
   yield key, el_value

Проверьте своё решение

In [175]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [176]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [177]:
import numpy as np

I, J, K = 2, 3, 40
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested):
    for iterable in nested:
        yield from iterable

def groupbykey(iterable):
    result = {}
    for k, v in iterable:
        result.setdefault(k, []).append(v)
    return result.items()

def RECORDREADER():
    yield [((0, i, j), small_mat[i, j]) for i in range(I) for j in range(J)]
    yield [((1, j, k), big_mat[j, k]) for j in range(J) for k in range(K)]

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    yield j if mat_num == 0 else i, (mat_num, i if mat_num == 0 else j, v1)

def REDUCE_JOIN(key, values):
    first_mat = [v for v in values if v[0] == 0]
    second_mat = [v for v in values if v[0] == 1]
    for f in first_mat:
        for s in second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapped = flatten(map(lambda kv: MAP(*kv), flatten(RECORDREADER())))
    reduced = flatten(map(lambda kv: REDUCE(*kv), groupbykey(mapped)))
    return reduced

intermediate = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(lambda: [intermediate], MAP_MUL, REDUCE_MUL)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")

MapReduce успешно выполнен!


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [178]:
import numpy as np
from collections import defaultdict

I, J, K = 2, 3, 40

maps = 4
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        yield from iterable

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = defaultdict(dict)
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            partitions[PARTITIONER(k2)][k2] = partitions[PARTITIONER(k2)].get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in partitions.items()]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print(f"{sum(len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions]))} key-value pairs were sent over a network.")
    return reduce_outputs

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def INPUTFORMAT():
    first_mat = [((0, i, j), val) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1]) for val in [small_mat[i, j]]]
    second_mat = [((1, j, k), val) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1]) for val in [big_mat[j, k]]]
    yield first_mat
    yield second_mat

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    if mat_num == 0:
        yield j, (mat_num, i, v1)
    else:
        yield i, (mat_num, j, v1)

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def GET_JOINED():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN)
joined = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL)
pre_result = [(partition_id, list(partition)) for partition_id, partition in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]
print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")


126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.06107093757285539)), ((0, 1), np.float64(0.15807310553239698)), ((0, 2), np.float64(0.03640767945461294)), ((0, 3), np.float64(0.24082081236598482)), ((0, 4), np.float64(0.2068187127378646)), ((0, 5), np.float64(0.136203093236136)), ((0, 6), np.float64(0.12895328390372102)), ((0, 7), np.float64(0.16547576462914718)), ((0, 8), np.float64(0.17689740136436774)), ((0, 9), np.float64(0.1947318133254978)), ((0, 10), np.float64(0.041373483238183725)), ((0, 11), np.float64(0.09892765470439939)), ((0, 12), np.float64(0.05781142590887277)), ((0, 13), np.float64(0.05804359123403802)), ((0, 14), np.float64(0.07531948375320476)), ((0, 15), np.float64(0.12856520900412904)), ((0, 16), np.float64(0.06296057124358993)), ((0, 17), np.float64(0.0927843430250817)), ((0, 18), np.float64(0.1366205468772308)), ((0, 19), np.float64(0.18307792319126745)), ((0, 20), np.float64(0.022651717214353773)), ((0, 21), np.float64(0.105418502382201

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [179]:
I, J, K = 2, 3, 40

maps = 3
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]

print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "Результат неверный!"
print("MapReduce успешно выполнен!")

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.6697308509308915)), ((0, 1), np.float64(0.48772623611276755)), ((0, 2), np.float64(0.19479281300505366)), ((0, 3), np.float64(0.5971374587950451)), ((0, 4), np.float64(0.5447487356266695)), ((0, 5), np.float64(0.40020937840843035)), ((0, 6), np.float64(0.4239067492165266)), ((0, 7), np.float64(0.31675184793482575)), ((0, 8), np.float64(0.6162437766888021)), ((0, 9), np.float64(0.002128008825899057)), ((0, 10), np.float64(0.1419883012167619)), ((0, 11), np.float64(0.25824977669459287)), ((0, 12), np.float64(0.1430649873255834)), ((0, 13), np.float64(0.5220653420787928)), ((0, 14), np.float64(0.4025605991033869)), ((0, 15), np.float64(0.3607847365593121)), ((0, 16), np.float64(0.11774114150013795)), ((0, 17), np.float64(0.7468468109256213)), ((0, 18), np.float64(0.7684520775359011)), ((0, 19), np.float64(0.23393712652800375)), ((0, 20), np.float64(0.34969374462022135)), ((0, 21), np.float64(0.7338545829026165)), ((